# 🛍 MarketEdge Retail Analytics — Profitability Insights
This notebook explores store-level sales and profitability trends for a mid-market retail chain using sample data.

### Objectives
- Identify margin leakage across stores
- Highlight top-performing stores and products
- Generate visuals for Tableau dashboard integration

In [ ]:
# 🧩 Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

In [ ]:
# 📥 Load Sample Dataset
sales = pd.read_csv('../data/sample_sales_data.csv')
sales.head()

In [ ]:
# 🧹 Data Cleaning
sales.drop_duplicates(inplace=True)
sales['date'] = pd.to_datetime(sales['date'])
sales.info()

In [ ]:
# 💰 Calculate Profit Margins
sales['margin'] = sales['revenue'] - sales['cost']
sales['margin_pct'] = (sales['margin'] / sales['revenue']) * 100
sales.describe()[['revenue', 'cost', 'margin', 'margin_pct']]

In [ ]:
# 📊 Pareto (80/20) Analysis — Stores Driving 80% of Total Margin
pareto = sales.groupby('store')['margin'].sum().reset_index().sort_values('margin', ascending=False)
pareto['cum_margin_pct'] = pareto['margin'].cumsum() / pareto['margin'].sum() * 100
pareto.head()

In [ ]:
# 🖼️ Visualization: Store Profitability Gradient Chart
plt.figure(figsize=(10,6))
colors = sns.color_palette('Spectral', len(pareto.head(10)))
bars = plt.bar(pareto.head(10)['store'], pareto.head(10)['margin'], color=colors, edgecolor='black')

for bar, value in zip(bars, pareto.head(10)['margin']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000, f"${value:,.0f}", ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.title('Store Profitability — Top 10 Stores by Margin', fontsize=16, fontweight='bold')
plt.xlabel('Store', fontsize=12)
plt.ylabel('Total Margin ($)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('../visuals/store_profitability_chart.png', dpi=300, bbox_inches='tight', transparent=False)
plt.show()

In [ ]:
# 💡 Key Insights Summary
top_20 = pareto[pareto['cum_margin_pct'] <= 80]
print(f"Number of stores driving 80% of total margin: {len(top_20)}")
print(f"Total margin from these stores: ${top_20['margin'].sum():,.2f}")
print('\nInsight: The top-performing stores account for the majority of profitability, confirming a strong 80/20 distribution.')